In [1]:
import os
import shutil
import argparse
import os
import sys



from keras_retinanet.preprocessing.csv_generator import CSVGenerator
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
from tensorflow import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time
from keras_retinanet.models import load_model

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# use this to change which GPU to use
# gpu = '0'

# # set the modified tf session as backend in keras
# setup_gpu(gpu)

2023-12-04 15:02:37.632757: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-04 15:02:37.632791: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-04 15:02:37.632816: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-04 15:02:37.639756: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-04 15:02:39.172403: W tensorflow/compiler/

In [5]:


# # folder_path = 'images/'


# # prefix = 'image'
# # file_extension = '.jpg'

# # num_duplicates = 49

# # for i, filename in enumerate(os.listdir(folder_path)):
# #     if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.jpeg'):
# #         new_filename = f'{i + 1}{file_extension}'

# #         shutil.copy(os.path.join(folder_path, filename), os.path.join(folder_path, new_filename))

# #         for j in range(num_duplicates):
# #             duplicate_filename = f'{i + 2 + j * len(os.listdir(folder_path))}{file_extension}'
# #             shutil.copy(os.path.join(folder_path, new_filename), os.path.join(folder_path, duplicate_filename))
# import os
# import random
# import csv
# # Replace this with the actual path to your directory
# base_directory = "valid/"

# # Create output directory if not exists
# output_directory = os.path.join(base_directory, "output")
# os.makedirs(output_directory, exist_ok=True)
# output_data = []
# # List of subdirectories (cancer and non_cancer)
# subdirectories = ["cancer", "non_cancer"]
# num_files = 10
# # Loop through each subdirectory
# subdirectories = ["cancer", "non_cancer"]

# # Number of files to generate
# num_files = 10

# # List to store generated data
# output_data = []

# # Loop through each subdirectory
# for subdirectory in subdirectories:
#     current_directory = os.path.join(base_directory, subdirectory)

#     # Check if the subdirectory exists
#     if os.path.exists(current_directory):

#         # Loop through each file in the subdirectory
#         for filename in os.listdir(current_directory):
#             # Generate random values
#             x1, y1 = random.randint(1, 400), random.randint(1, 400)
#             x2, y2 = random.randint(x1+20, 500), random.randint(y1+20, 500)

#             # Create the output string with the original filename
#             output_string = f"valid/{subdirectory}/{filename},{x1},{y1},{x2},{y2},{subdirectory}"

#             # Create the output string with the original filename

#             # Append the data to the list
#             output_data.append(output_string.split(','))


# # Write the output to a CSV file
# csv_file_path = os.path.join('./', "output.csv")
# with open(csv_file_path, mode='w', newline='') as csv_file:
#     csv_writer = csv.writer(csv_file)
#     csv_writer.writerows(output_data)

### Post Training Quantization


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

In [3]:
model_keras = load_model(
    '../PyAVE_cmu_fall2023/cmu_fall2023/assets/resnet50_csv_26.h5', backbone_name='resnet50')
new_input_shape = (1067, 800, 3)
model_keras.inputs[0].set_shape((None, *new_input_shape))
model_keras = models.convert_model(model_keras)
# Rebuild the model to apply the changes
model_keras = keras.models.Model(
    inputs=model_keras.inputs, outputs=model_keras.outputs)

2023-12-04 15:02:47.548719: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-04 15:02:47.799758: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-04 15:02:47.799953: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [ ]:
model_keras.summary()

In [7]:
import io
summary = io.StringIO()
model_keras.summary(print_fn=lambda x: summary.write(x + '\n'))
model_summary = summary.getvalue()
summary.close()
with open('model_summary.txt', 'w') as file:
    file.write(model_summary)

print("Model summary written to 'model_summary.txt'")

Model summary written to 'model_summary.txt'


In [ ]:
from keras.models import Model

# Load your model
model_keras = load_model('../PyAVE_cmu_fall2023/cmu_fall2023/assets/resnet50_csv_26.h5', backbone_name='resnet50')

# Identify the output of the last layer you want to keep
# For example, if 'res5c_relu' is the last feature extraction layer and 'classification' is your final classification layer
feature_extraction_output = model_keras.get_layer('res5c_relu').output
classification_output = model_keras.get_layer('classification').output

# Create a new model
new_model = Model(inputs=model_keras.input, outputs=[feature_extraction_output, classification_output])

# Save or further process your model
new_model.summary()


In [32]:
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.layers import GlobalAveragePooling2D
from keras.layers import Input

from keras.models import Model


# Split the model after 'res5c_relu'
feature_extraction_output = model_keras.get_layer('res5c_relu').output
feature_extraction_model = Model(inputs=model_keras.input, outputs=feature_extraction_output)

# For the remaining model, create a new input layer
input_shape = model_keras.get_layer('C5_reduced').input_shape[1:]
new_input = Input(shape=input_shape)

# Reconnect the layers from 'C5_reduced' onwards
prev_layer = new_input
for layer in model_keras.layers[model_keras.layers.index(model_keras.get_layer('C5_reduced')):]:
    prev_layer = layer(prev_layer)

remaining_model = Model(new_input, prev_layer)

# Save the models
feature_extraction_model.save('feature_extraction_model.h5')
remaining_model.save('remaining_model.h5')


OperatorNotAllowedInGraphError: Exception encountered when calling layer "P5_upsampled" (type UpsampleLike).

in user code:

    File "/home/developer/.local/lib/python3.10/site-packages/keras_retinanet/layers/_misc.py", line 102, in call  *
        source, target = inputs

    OperatorNotAllowedInGraphError: Iterating over a symbolic `tf.Tensor` is not allowed. You can attempt the following resolutions to the problem: If you are running in Graph mode, use Eager execution mode or decorate this function with @tf.function. If you are using AutoGraph, you can try decorating this function with @tf.function. If that does not work, then you may be using an unsupported feature or your source code may not be visible to AutoGraph. See https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/g3doc/reference/limitations.md#access-to-source-code for more information.


Call arguments received by layer "P5_upsampled" (type UpsampleLike):
  • inputs=tf.Tensor(shape=(None, Non

In [ ]:
new_model.summary()

In [ ]:
from tensorflow.keras.models import  Model


# Load the model
model_keras = load_model(
    '../PyAVE_cmu_fall2023/cmu_fall2023/assets/resnet50_csv_26.h5', backbone_name='resnet50')

# Modify the input shape
new_input_shape = (1067, 800, 3)
model_keras.inputs[0].set_shape((None, *new_input_shape))
model_keras = models.convert_model(model_keras)

# Rebuild the model to apply the changes
# model_keras = Model(inputs=model_keras.inputs, outputs=model_keras.outputs)

# Assuming your original model is named 'original_model'
# Replace 'regression_submodel' and 'classification_submodel' with the actual names of these layers in your model
original_model = model_keras

# Find the index of the layers you want to keep up to
last_layer_to_keep = 'res5c_relu'  # Replace with the actual name of the last layer you want to keep
index_to_keep = original_model.layers.index(original_model.get_layer(last_layer_to_keep))

# Create a new model with layers up to the specified index
new_model1 = Model(inputs=original_model.inputs, outputs=original_model.layers[index_to_keep + 1].output)

# Copy weights only for the layers you decided to keep
for layer in new_model1.layers:
    layer_name = layer.name
    weights = original_model.get_layer(layer_name).get_weights()
    # Set the weights in the corresponding layer of the new model
    layer.set_weights(weights)

# Display the summary of the new model
new_model1.summary()


In [9]:
fin_model = keras.Sequential()

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

your_model = new_model1
# Get the output layer of your model
output_layer = your_model.get_layer('res5c_relu').output

# Add Global Average Pooling layer
pooled_output = GlobalAveragePooling2D()(output_layer)

# Add Dense layer for binary classification
binary_classification_output = Dense(1, activation='sigmoid')(pooled_output)

# Create a new model with the modified output
binary_classification_model = Model(inputs=your_model.input, outputs=binary_classification_output)

# Display the summary of the new model
binary_classification_model.summary()

In [ ]:
for layer in binary_classification_model.layers[:-2]:
    layer_name = layer.name
    weights = model_keras.get_layer(layer_name).get_weights()
    # Set the weights in the corresponding layer of the new model
    layer.set_weights(weights)
    # Set the layer as non-trainable
    layer.trainable = False

# Display the summary of the new model
binary_classification_model.summary()

In [7]:
binary_classification_model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')

In [6]:
import pandas as pd
from PIL import Image
import numpy as np

# Load the CSV file
csv_path = 'output copy.csv'  # Replace with the actual path to your CSV file
df = pd.read_csv(csv_path, header=None, names=['path', 'x1', 'y1', 'x2', 'y2', 'label'])

# Define the image size you want to use
image_size = (1067, 800)  # Adjust the size based on your requirements

# Initialize numpy arrays
x_train = np.zeros((len(df), *image_size, 3), dtype=np.uint8)
y_train = np.zeros((len(df),), dtype=np.int32)

# Load images and labels
for i, row in df.iterrows():
    image_path = row['path']
    label = row['label']

    # Load image and resize
    image = Image.open(image_path).convert('RGB')
    image = image.rotate(90)

    image = np.asarray(image)
    image = image[:, :, ::-1].copy()

    image = preprocess_image(image)
    image, scale = resize_image(image)

    # Convert image to numpy array and normalize
    x_train[i] = np.array(image)
    y_train[i] = label



# Display the shapes of x_train and y_train
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)



x_train shape: (10, 1067, 800, 3)
y_train shape: (10,)


In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = ""
binary_classification_model.fit(x_train,y_train, epochs=20)

In [27]:
image = Image.open(
    '../Validation/images/MJ-01323_2022_06_22_10_48.jpg').convert('RGB')
image = image.rotate(90)

image = np.asarray(image)
image = image[:, :, ::-1].copy()

image = preprocess_image(image)
image, scale = resize_image(image)
print(image.shape)
# Run the inference
start = time.time()
scores = new_model.predict_on_batch(np.expand_dims(image, axis=0))
inference_time = time.time() - start
print(inference_time)
# predicted_label = labels[0][np.argmax(scores[0])]
# confidence = scores[0][np.argmax(scores[0])]

(1067, 800, 3)
1.0405757427215576


2023-12-04 15:46:07.713497: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.16GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


In [30]:
scores[0]

array([0.5456725 , 0.45432755], dtype=float32)

0.046861164

In [18]:
converter = tf.lite.TFLiteConverter.from_keras_model(new_model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmplf392zzu/assets


INFO:tensorflow:Assets written to: /tmp/tmplf392zzu/assets
2023-12-04 15:20:25.026689: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-12-04 15:20:25.026725: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-12-04 15:20:25.029613: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmplf392zzu
2023-12-04 15:20:25.052500: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-12-04 15:20:25.052549: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmplf392zzu
2023-12-04 15:20:25.093643: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled
2023-12-04 15:20:25.118410: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-12-04 15:20:25.708032: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

In [10]:
tflite_model_file = tflite_models_dir/"/home/developer/Documents/CMU/Capstone_Codes/cervical-cancer-project/models/tflite/avl_ghl_classify.tflite"
tflite_model_file.write_bytes(tflite_model)

93967364

In [11]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

In [12]:
tflite_fp16_model = converter.convert()
tflite_model_fp16_file = tflite_models_dir/"/home/developer/Documents/CMU/Capstone_Codes/cervical-cancer-project/models/tflite/avl_ghl_fp16_classify.tflite"
tflite_model_fp16_file.write_bytes(tflite_fp16_model)

INFO:tensorflow:Assets written to: /tmp/tmp0thohv27/assets


INFO:tensorflow:Assets written to: /tmp/tmp0thohv27/assets
2023-11-28 08:54:41.886772: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-28 08:54:41.886796: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-28 08:54:41.887007: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp0thohv27
2023-11-28 08:54:41.913498: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-28 08:54:41.913524: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp0thohv27
2023-11-28 08:54:41.977312: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-11-28 08:54:42.499635: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmp0thohv27
2023-11-28 08:54:42.682018: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

47014384

In [19]:
def create_generator():
    """ Create generators for evaluation.
    """
    common_args = {

        'preprocess_image': preprocess_image,

    }

    validation_generator = CSVGenerator(
        'output copy.csv',
        'class.csv',
        shuffle_groups=False,
        **common_args
    )

    return validation_generator

In [20]:
import pathlib

tflite_models_dir = pathlib.Path("/tmp/mnist_tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

In [21]:
gen = create_generator()
# create an image generator with a batch size of 1
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_image)
test_generator = gen


def represent_data_gen():
    """ it yields an image one by one """
    for ind in range(len(gen.image_data.keys())):
        # it returns (image and label) tuple
        image = test_generator.load_image(ind)
        image = image[:, :, ::-1].copy()
        image = preprocess_image(image)
        image, scale = resize_image(image)

        print(image.shape)
        yield [np.expand_dims(image, axis=0)]  # return only image

In [22]:
converter = tf.lite.TFLiteConverter.from_keras_model(new_model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = represent_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpxafehf3e/assets


INFO:tensorflow:Assets written to: /tmp/tmpxafehf3e/assets
/home/developer/.local/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:947: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2023-12-04 15:21:05.100218: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-12-04 15:21:05.100246: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-12-04 15:21:05.100427: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpxafehf3e
2023-12-04 15:21:05.120936: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-12-04 15:21:05.120965: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpxafehf3e
2023-12-04 15:21:05.182115: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-12-04 15:21:05

(1067, 800, 3)
(1067, 800, 3)
(1067, 800, 3)
(1067, 800, 3)
(1067, 800, 3)
(1067, 800, 3)
(1067, 800, 3)
(1067, 800, 3)
(1067, 800, 3)
(1067, 800, 3)
(1067, 800, 3)
(1067, 800, 3)


fully_quantize: 0, inference_type: 6, input_inference_type: UINT8, output_inference_type: UINT8


In [24]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model_keras)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = represent_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant = converter.convert()

KeyboardInterrupt: 

In [23]:
tflite_model_quant_file = tflite_models_dir / \
    "/home/developer/Documents/CMU/Capstone_Codes/cervical-cancer-project/models/tflite/new_1.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

34820200

In [24]:
interpreter = tf.lite.Interpreter(model_path=str("/home/developer/Documents/CMU/Capstone_Codes/cervical-cancer-project/models/tflite/avl_classify.tflite"))
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [25]:
interpreter.get_input_details()

[{'name': 'serving_default_input_1:0',
  'index': 0,
  'shape': array([   1, 1067,  800,    3], dtype=int32),
  'shape_signature': array([  -1, 1067,  800,    3], dtype=int32),
  'dtype': numpy.uint8,
  'quantization': (1.0774157047271729, 115),
  'quantization_parameters': {'scales': array([1.0774157], dtype=float32),
   'zero_points': array([115], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [26]:
interpreter.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 203,
  'shape': array([1, 1], dtype=int32),
  'shape_signature': array([-1,  1], dtype=int32),
  'dtype': numpy.uint8,
  'quantization': (0.00390625, 0),
  'quantization_parameters': {'scales': array([0.00390625], dtype=float32),
   'zero_points': array([0], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [27]:
input_index = interpreter.get_input_details()[0]["index"]
image = Image.open(
    '../Validation/images/SN-05037_2022_05_30_12_50.jpg').convert('RGB')


image = image.rotate(90)

image = np.asarray(image)
image = image[:, :, ::-1].copy()

image = preprocess_image(image)
image, scale = resize_image(image)
print(image.shape)
# Run the inference
# start = time.time()
# boxes, scores, labels = model_keras.predict_on_batch(np.expand_dims(image, axis=0))
# inference_time = time.time() - start
# predicted_label = labels[0][np.argmax(scores[0])]
# confidence = scores[0][np.argmax(scores[0])]
image = image.astype(np.uint8)
# print(image[0])
interpreter.set_tensor(input_index, np.reshape(image, ((1, 1067,  800,    3))))

# Run inference.
start = time.time()
interpreter.invoke()

# Post-processing: remove batch dimension and find the digit with highest
# probability.
# output = interpreter.tensor(output_index)
# print(len(output))
# print(output.shape)

scores = interpreter.get_tensor(interpreter.get_output_details()[0]["index"])

# scores = interpreter.get_tensor(interpreter.get_output_details()[1]["index"])
inference_time = time.time() - start
print(inference_time)
# predicted_label = labels[0][np.argmax(scores[0])]
# confidence = scores[0][np.argmax(scores[0])]
#..\..\..\images\Cervical\Zohreh 2023-01-27\images\SN-05037_2022_05_30_12_50.jpg

(1067, 800, 3)
2.2862095832824707


In [28]:
scores

array([[128]], dtype=uint8)

In [55]:
scores[0][160736]

array([  3, 158], dtype=uint8)

In [18]:
model_keras.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1067, 800, 3)]       0         []                            
                                                                                                  
 padding_conv1 (ZeroPadding  (None, None, None, 3)        0         ['input_1[0][0]']             
 2D)                                                                                              
                                                                                                  
 conv1 (Conv2D)              (None, None, None, 64)       9408      ['padding_conv1[0][0]']       
                                                                                                  
 bn_conv1 (BatchNormalizati  (None, None, None, 64)       256       ['conv1[0][0]']           

In [23]:
model_keras.get_layer("classification_submodel")

In [19]:
from tensorflow.keras.models import Model
# Replace with the actual name of the last layer you want to keep
last_layer_to_keep = 'C6_relu'
index_to_keep = model_keras.layers.index(
    model_keras.get_layer(last_layer_to_keep))

# Create a new model with layers up to the specified index
new_model = Model(inputs=model_keras.inputs,
                  outputs=model_keras.layers[index_to_keep + 1].output)

# Display the summary of the new model
new_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1067, 800, 3)]       0         []                            
                                                                                                  
 padding_conv1 (ZeroPadding  (None, None, None, 3)        0         ['input_1[0][0]']             
 2D)                                                                                              
                                                                                                  
 conv1 (Conv2D)              (None, None, None, 64)       9408      ['padding_conv1[0][0]']       
                                                                                                  
 bn_conv1 (BatchNormalizati  (None, None, None, 64)       256       ['conv1[0][0]']         

In [24]:
from tensorflow.keras.models import Model
# Replace with the actual name of the last layer you want to keep
last_layer_to_keep = 'classification_submodel'
index_to_keep = model_keras.layers.index(
    model_keras.get_layer(last_layer_to_keep))

# Create a new model with layers up to the specified index
new_model2 = Model(inputs=model_keras.inputs,
                   outputs=model_keras.layers[index_to_keep + 1].output)

# Display the summary of the new model
new_model2.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1067, 800, 3)]       0         []                            
                                                                                                  
 padding_conv1 (ZeroPadding  (None, None, None, 3)        0         ['input_1[0][0]']             
 2D)                                                                                              
                                                                                                  
 conv1 (Conv2D)              (None, None, None, 64)       9408      ['padding_conv1[0][0]']       
                                                                                                  
 bn_conv1 (BatchNormalizati  (None, None, None, 64)       256       ['conv1[0][0]']         

In [25]:
converter = tf.lite.TFLiteConverter.from_keras_model(new_model2)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = represent_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpdc65rdp_/assets


INFO:tensorflow:Assets written to: /tmp/tmpdc65rdp_/assets
/home/developer/.local/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2023-11-26 12:22:56.595174: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-26 12:22:56.595197: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-26 12:22:56.595362: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpdc65rdp_
2023-11-26 12:22:56.615800: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-26 12:22:56.615830: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpdc65rdp_
2023-11-26 12:22:56.675360: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-11-26 12:22:57

(1067, 800, 3)
(1067, 800, 3)
(1067, 800, 3)
(1067, 800, 3)
(1067, 800, 3)
(1067, 800, 3)
(1067, 800, 3)
(1067, 800, 3)
(1067, 800, 3)
(1067, 800, 3)
(1067, 800, 3)
(1067, 800, 3)


fully_quantize: 0, inference_type: 6, input_inference_type: UINT8, output_inference_type: UINT8


In [26]:
tflite_model_quant_file = tflite_models_dir / \
    "/home/developer/Documents/CMU/Capstone_Codes/cervical-cancer-project/models/tflite/part2_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

34856968

In [33]:

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

# Assuming your original model is named 'original_model'
# Replace 'C6_relu' with the actual name of this layer in your model
original_model = model_keras
# Find the index of the layer 'C6_relu'
c6_relu_index = original_model.layers.index(
    original_model.get_layer('C6_relu'))

# Create the shared layer up to 'C6_relu'
shared_layer = Model(inputs=original_model.input,
                     outputs=original_model.layers[c6_relu_index].output,
                     name='shared_layer')

# Define the input for the second model
input_for_second_model = Input(
    shape=shared_layer.output_shape[1:], name='input_for_second_model')

# Create the first model using the shared layer
model_up_to_c6_relu = shared_layer

# # Create the second model starting from 'C6_relu'
# output_of_second_model = original_model.layers[-1].output
# model_from_c6_relu = Model(inputs=input_for_second_model,
#                            outputs=output_of_second_model,
#                            name='model_from_c6_relu')

# # Display the summary of the first model
model_up_to_c6_relu.summary()

Model: "shared_layer"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 padding_conv1 (ZeroPadding  (None, None, None, 3)        0         ['input_1[0][0]']             
 2D)                                                                                              
                                                                                                  
 conv1 (Conv2D)              (None, None, None, 64)       9408      ['padding_conv1[0][0]']       
                                                                                                  
 bn_conv1 (BatchNormalizati  (None, None, None, 64)       256       ['conv1[0][0]']    

 ization)                                                                                         
                                                                                                  
 res2a_branch2b_relu (Activ  (None, None, None, 64)       0         ['bn2a_branch2b[0][0]']       
 ation)                                                                                           
                                                                                                  
 res2a_branch2c (Conv2D)     (None, None, None, 256)      16384     ['res2a_branch2b_relu[0][0]'] 
                                                                                                  
 res2a_branch1 (Conv2D)      (None, None, None, 256)      16384     ['pool1[0][0]']               
                                                                                                  
 bn2a_branch2c (BatchNormal  (None, None, None, 256)      1024      ['res2a_branch2c[0][0]']      
 ization) 